# Data Cleaning Notebook

## Objectives

*   Evaluate missing data
*   Clean data

## Inputs

* outputs/datasets/collection/HeritageHousing.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned

## Conclusions

  * Cleaned Data Pipeline

---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/HeritageHousing.csv")
    )
df.head(5)

---

# Predictive Power Score and Correlation Study

Let's run some more analysis on the data before we continue, so we can look at how the missing data correlates with the rest of our data set.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

def heatmap_corr(df,threshold, figsize=(20,12), font_annot = 8):
  if len(df.columns) > 1:
    mask = np.zeros_like(df, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True

    fig, axes = plt.subplots(figsize=figsize)
    sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                linewidth=0.5
                     )
    axes.set_yticklabels(df.columns, rotation = 0)
    plt.ylim(len(df.columns),0)
    plt.show()


def heatmap_pps(df,threshold, figsize=(20,12), font_annot = 8):
    if len(df.columns) > 1:

      mask = np.zeros_like(df, dtype=np.bool)
      mask[abs(df) < threshold] = True

      fig, ax = plt.subplots(figsize=figsize)
      ax = sns.heatmap(df, annot=True, xticklabels=True,yticklabels=True,
                       mask=mask,cmap='rocket_r', annot_kws={"size": font_annot},
                       linewidth=0.05,linecolor='grey')
      
      plt.ylim(len(df.columns),0)
      plt.show()



def CalculateCorrAndPPS(df):
  df_corr_spearman = df.corr(method="spearman")
  df_corr_pearson = df.corr(method="pearson")

  pps_matrix_raw = pps.matrix(df)
  pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

  pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
  print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
  print(pps_score_stats.round(3))

  return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix,CorrThreshold,PPS_Threshold,
                      figsize=(20,12), font_annot=8 ):

  print("\n")

  print("\n")
  print("*** Heatmap: Spearman Correlation ***")
  print("It evaluates monotonic relationship \n")
  heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Pearson Correlation ***")
  print("It evaluates the linear relationship between two continuous variables \n")
  heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Power Predictive Score (PPS) ***")
  print(f"PPS detects linear or non-linear relationships between two columns.\n")
  heatmap_pps(df=pps_matrix,threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

In [ ]:
DisplayCorrAndPPS(df_corr_pearson=df_corr_pearson,
                  df_corr_spearman=df_corr_spearman, 
                  pps_matrix=pps_matrix,
                  CorrThreshold=0.6, PPS_Threshold=0.15,
                  figsize=(10,10), font_annot=8)

---

## Missing Data

Let's remind ourselves of the columns that have missing data.

In [ ]:
def EvaluateMissingData(df):
  missing_data_absolute = df.isnull().sum()
  missing_data_percentage = round(missing_data_absolute/len(df)*100 , 2)
  df_missing_data = (pd.DataFrame(
                          data= {"RowsWithMissingData": missing_data_absolute,
                                 "PercentageOfDataset": missing_data_percentage,
                                 "DataType":df.dtypes}
                                  )
                    .sort_values(by=['PercentageOfDataset'],ascending=False)
                    .query("PercentageOfDataset > 0")
                    )

  return df_missing_data

In [ ]:
EvaluateMissingData(df)

Now we can discuss what changes we want to make to each of the variables and their missing data.

First we will want to use an ArbitraryNumberImputer function to replace the missing data of `2ndFlrSF`, `EnclosedPorch`, `MasVnrArea` and `WoodDeckSF` with 0 as it is the most common value for all of these variables. It's safer to assume that a missing value for these variables means that the house simply doesn't have one of those features. For example, `EnclosedPorch` is missing 90.7% and 7.9% of the data that we do have is 0 meaning that for more than 80% of the data that we do have the value is 0.

In most cases it's fine to drop columns that are missing 80% or more of their data. But we know that a houses size is one of the greatest predictors of Sale Price and want to see if we can gain a better PPS by combining all the variables related to the houses size once the data is cleaned.

The other variable related to the size of the house is `LotFrontage`. With the pandas profile report in the last notebook, we can see that this data is much closer to being normally distributed and so, we will use the MeanMedianImputer function to impute the NaN values with a median value.

---

Next is `Bedroom Above Grade`. We don't know what the 'grade' is and this column is missing just a bit more information than we would like to be able to drop all the rows. Perhaps with more understanding of what the variable is, we could estimate the number of Bedrooms above grade based on other houses vs their overall size and perhaps quality. For now though we'll use the MeanMedianImputer. 

---

Then there are two columns of missing data for the garages, `GarageFinish` and `GarageYrBlt`. Let's take one more look at them, along with `YearBuilt`(of the house) and `GarageArea`.

In [ ]:
df_nan_finish = df[df['GarageFinish'].isna()]
df_nan_finish[['GarageFinish','GarageYrBlt', 'GarageArea', 'YearBuilt']]

In [ ]:
df_nan_year = df[df['GarageYrBlt'].isna()]
df_nan_year[['GarageFinish','GarageYrBlt', 'GarageArea']]

And two last things to check, `GarageFinish` and `BasementFinish` where the value is missing and the area of the related feature is 0.

In [ ]:
df_compare1 = df.loc[(df['GarageFinish'].isna()) & (df['GarageArea'] == 0)]
df_compare1[['GarageFinish', 'GarageArea']]

In [ ]:
df_compare2 = df.loc[(df['BsmtFinType1'].isna()) & (df['TotalBsmtSF'] == 0)]
df_compare2[['BsmtFinType1', 'TotalBsmtSF']]

There doesn't seem to be any clear indicator of what the best way to set the `GarageFinish` as would be. 
It also showed little to no correlation against the sale price in our initial study of the data we can fill the missing data with `Unf`. Some of the `GarageFinish` missing values do share a `GarageArea` of 0, which means ideally we would for these cases fill them with `None` instead. But as it's only for 8 of the missing values we're happy to skip over this for now.
Using the PPS we see there is moderate correlation between `GarageFinish`, `Overall Quality`, `YearBuilt` and others. So in future we could perhaps predict what `GarageFinish` should be based on other variables to improve our model. But for now we're happy to use the frequent category as our imputation method by using the CategoricalVariableImputer function. 

---

We'll use the same reasoning and imputation method as `GarageFinish` for `BsmtFinType1`, using `Unf` for any missing data. We dont need to fill any of the values with `None` for this variable, as we checked that all the missing values do have a `TotalBsmtSF` 

---

And for `GarageYrBlt` we can see the NaN values are appearing where a house doesn't have a garage. 
We can't set this value to 0 but there's a clear indication that most houses are built with a garage as they typically have the same `YearBuilt` and `GarageYrBlt` dates.
So for this we'll replace any missing values with the date the year the house was built.

![DataCleaningSpreadsheet](../media/DataCleaning.png)

Before we clean the data, we will use a function built by Code Institue. This will perform the imputations using imputation methods from feature engine and then plot the original and cleaned datasets against each other to help visualise the effect our changes will have on the data.

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

Now we set up the parameters for the cleaning functions and run them. First is the AbritraryNumberImputer.

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer

variables_method = ['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']
variables_method

imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

Next is the `LotFrontage` and `BedroomAbvGr` for which we use the MeanMedianImputer.

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_method = ['LotFrontage', 'BedroomAbvGr']
variables_method

imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

The CategoricalImputer for `GarageFinish` and `BsmtFinType1`

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_method = ['GarageFinish', 'BsmtFinType1']
variables_method

imputer = CategoricalImputer(imputation_method='missing',fill_value='Unf',
                             variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

And last, the `GarageYrBlt`. We don't have a feature engine imputer for this one so we will have to use a rudimentary way to show the difference. As we mentioned, the `GarageYrBlt` is almost always the same as the `YearBuilt`. So with this variable and imputation method the difference in change is difficult to see.

In [ ]:
before_garage_impute = df
df['GarageYrBlt'].fillna(df['YearBuilt'])

fig = plt.figure()
ax = fig.add_subplot(111)
before_garage_impute['GarageYrBlt'].plot(kind='kde', ax=ax, linewidth=5)
df['GarageYrBlt'].plot(kind='kde', ax=ax, color='red')
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')

---

## Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

---

## Apply the Changes to the Missing Data

If we were removing any rows from our data, we would also check the data again during this stage as it would likely alter the distribution on some of the variables we are cleaning. 
As this is not the case, we can be sure that our analysis of how the data changes from cleaning will remain the same as our earlier study.

In [ ]:
variables_method = ['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']
imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
variables_method = ['LotFrontage', 'BedroomAbvGr']
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
variables_method = ['GarageFinish', 'BsmtFinType1']
imputer = CategoricalImputer(imputation_method='missing',fill_value='Unf',
                             variables=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
TrainSet['GarageYrBlt'].fillna(df['YearBuilt'], inplace=True)
TestSet['GarageYrBlt'].fillna(df['YearBuilt'], inplace=True)

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

In [ ]:
df_missing_data = EvaluateMissingData(TestSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

---

# Push files to Repo

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)